<a href="https://colab.research.google.com/github/Abhinavl3v3l/EVA5/blob/master/(MNIST99_4)EVA5_Session_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 12, 3,)  #input = 28 Output = 26 RF = 3
        self.bn1 = nn.BatchNorm2d(12)
        self.do1 = nn.Dropout(0.3)
                              

        self.conv2 = nn.Conv2d(12, 20, 3) #input = 26 Output = 24 RF = 5
        self.bn2 = nn.BatchNorm2d(20)
        self.do2 = nn.Dropout(0.1)

        self.one1 = nn.Conv2d(20, 10, 1)  #input = 24 Output = 24 RF = 5
        self.pool1 = nn.MaxPool2d(2, 2)   #input = 24 Output = 12 RF = 10

        self.conv3 = nn.Conv2d(10, 16, 3) #input = 12 Output = 10 RF = 12
        self.bn3 = nn.BatchNorm2d(16)
        self.do3 = nn.Dropout(0.1)
        
        self.conv4 = nn.Conv2d(16, 16, 3)  #input = 10 Output = 8 RF = 14
        self.bn4 = nn.BatchNorm2d(16)
        self.do4 = nn.Dropout(0.1)

        # self.pool2 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(16, 16, 3) #input = 8 Output = 6 RF = 16
        self.bn5 = nn.BatchNorm2d(16)
        self.do5 = nn.Dropout(0.1)
                              
        self.conv6 = nn.Conv2d(16, 16, 3) #input = 6 Output = 4 RF = 18
        self.bn6 = nn.BatchNorm2d(16)
        self.do6 = nn.Dropout(0.1)
        
        self.conv7 = nn.Conv2d(16, 16, 3) #input = 4 Output = 2 RF = 20
        self.bn7 = nn.BatchNorm2d(16)
        self.do7 = nn.Dropout(0.1)

        self.conv8 = nn.Conv2d(16, 10, 2) #input = 2 Output = 1 RF = 22

    def forward(self, x):
        x = self.pool1(self.one1(self.do2(self.bn2(F.relu(self.conv2(self.do1(self.bn1(F.relu(self.conv1(x))))))))))
        x = self.do4(self.bn4(F.relu(self.conv4(self.do3(self.bn3(F.relu(self.conv3(x))))))))
        x = self.do6(self.bn6(F.relu(self.conv6(self.do5(self.bn5(F.relu(self.conv5(x))))))))
        x = self.conv8(self.do7(self.bn7(F.relu(self.conv7(x)))))
        # print(x.shape)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 12, 26, 26]             120
       BatchNorm2d-2           [-1, 12, 26, 26]              24
           Dropout-3           [-1, 12, 26, 26]               0
            Conv2d-4           [-1, 20, 24, 24]           2,180
       BatchNorm2d-5           [-1, 20, 24, 24]              40
           Dropout-6           [-1, 20, 24, 24]               0
            Conv2d-7           [-1, 10, 24, 24]             210
         MaxPool2d-8           [-1, 10, 12, 12]               0
            Conv2d-9           [-1, 16, 10, 10]           1,456
      BatchNorm2d-10           [-1, 16, 10, 10]              32
          Dropout-11           [-1, 16, 10, 10]               0
           Conv2d-12             [-1, 16, 8, 8]           2,320
      BatchNorm2d-13             [-1, 16, 8, 8]              32
          Dropout-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.07, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0503, Accuracy: 9838/10000 (98.38%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0367, Accuracy: 9886/10000 (98.86%)



...


Test set: Average loss: 0.0184, Accuracy: 9942/10000 (99.42%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0210, Accuracy: 9937/10000 (99.37%)



loss=0.02396484650671482 batch_id=468: 100%|██████████| 469/469 [00:13<00:00, 35.85it/s]



Test set: Average loss: 0.0184, Accuracy: 9951/10000 (99.51%)

